In [1]:
import dask
import dask.threaded
import dask.multiprocessing
from dask.distributed import Client

client = Client()
client

/scratch/cnt0024/hmg2840/albert7a/DEV/git/conda-pack/pytide/lib/python3.6/site-packages/distributed/node.py:244: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 41576 instead
  http_address["port"], self.http_server.port


Client Scheduler: tcp://127.0.0.1:45686 Dashboard: http://127.0.0.1:41576/status,Cluster Workers: 8 Cores: 56 Memory: 270.19 GB


In [2]:
print(len(client.scheduler_info()["workers"]))

8


In [4]:
import zarr
import netCDF4
import pytide
import xarray as xr
import os
import numpy as np

In [5]:
%%time
ds=xr.open_zarr('/store/albert7a/eNATL60/zarr/eNATL60-BLBT02-SSH-1h')
ds_sorted=ds.sortby('time_counter')


CPU times: user 658 ms, sys: 82 ms, total: 740 ms
Wall time: 1.82 s


In [6]:
START_DATE = np.datetime64('2009-07-01')
END_DATE = np.datetime64('2009-09-30')
time_series=ds_sorted['time_counter']
period = (time_series >= START_DATE) & (time_series <= END_DATE)
time=time_series[period]
ssh=ds_sorted.sossheig[period,:,0:2785]
t=time.values

In [8]:
%%time
wt = pytide.WaveTable()
wt = pytide.WaveTable(["M2", "S2", "N2", "O1", "K1"])
f, vu = wt.compute_nodal_modulations(t)

CPU times: user 1.66 ms, sys: 0 ns, total: 1.66 ms
Wall time: 972 µs


In [9]:
def dask_array_rechunk(da, axis=0):
    """Search for the optimal block cutting without modifying the axis 'axis'
    in order to optimize its access in memory."""
    nblocks = 1
    
    def calculate_chuncks_size(chunks, size):
        result = np.array(chunks).prod() * size
        return result / (1000**2)
       
    while True:
        chunks = []
        div = int(np.sqrt(nblocks))
        for index, item in enumerate(da.chunks):
            chunks.append(np.array(item).sum() * (div if index == axis else 1))
        chunks = tuple(item // div for index, item in enumerate(chunks))
        chuncks_size = calculate_chuncks_size(chunks, da.dtype.itemsize)
        if chuncks_size > 100 and chuncks_size < 150:
            return chunks
        nblocks += 1

In [10]:
def _apply_along_axis(arr, func1d, func1d_axis, func1d_args, func1d_kwargs):
    """Wrap apply_along_axis"""
    return np.apply_along_axis(func1d, func1d_axis, arr, *func1d_args,
                                  **func1d_kwargs)


def apply_along_axis(func1d, axis, arr, *args, **kwargs):
    """Apply the harmonic analysis to 1-D slices along the given axis."""
    arr = dask.array.core.asarray(arr)

    # Validate and normalize axis.
    arr.shape[axis]
    axis = len(arr.shape[:axis])

    # Rechunk so that analyze is applied over the full axis.
    arr = arr.rechunk(arr.chunks[:axis] + (arr.shape[axis:axis + 1], ) +
                      arr.chunks[axis + 1:])

    # Test out some data with the function.
    test_data = np.ones(args[0].shape[1], dtype=arr.dtype)
    test_result = np.array(func1d(test_data, *args, **kwargs))

    # Map analyze over the data to get the result
    # Adds other axes as needed.
    result = arr.map_blocks(
        _apply_along_axis,
        name=dask.utils.funcname(func1d) + '-along-axis',
        dtype=test_result.dtype,
        chunks=(arr.chunks[:axis] + test_result.shape + arr.chunks[axis + 1:]),
        drop_axis=axis,
        new_axis=list(range(axis, axis + test_result.ndim, 1)),
        func1d=func1d,
        func1d_axis=axis,
        func1d_args=args,
        func1d_kwargs=kwargs,
    )

    return result

In [11]:
dask_array_rechunk(ssh)

(720, 168, 298)

In [ ]:
ssh_rechunk = ssh.chunk(dask_array_rechunk(ssh))
future = apply_along_axis(pytide.WaveTable.harmonic_analysis, 0, ssh_rechunk,
                          *(f, vu))


In [13]:
len(future.dask)

209271

In [12]:
%%time
analysis = future.compute()

CPU times: user 4min 25s, sys: 2min 59s, total: 7min 24s
Wall time: 2h 14min 29s


In [14]:
%%time
amp_da=xr.DataArray(np.absolute(analysis),dims={'freq','y','x'},name='tidal-amp')
amp_da.to_netcdf(path='/scratch/cnt0024/hmg2840/albert7a/eNATL60/eNATL60-BLBT02-S/1h/tide/pytide/eNATL60tiersX1-BLBT02_tidal_amp_y2009m07-m09.nc',engine='scipy')

phase_da=xr.DataArray(np.angle(analysis),dims={'freq','y','x'},name='tidal-phase')
phase_da.to_netcdf(path='/scratch/cnt0024/hmg2840/albert7a/eNATL60/eNATL60-BLBT02-S/1h/tide/pytide/eNATL60tiersX1-BLBT02_tidal_phase_y2009m07-m09.nc',engine='scipy')

CPU times: user 8.61 s, sys: 8.2 s, total: 16.8 s
Wall time: 22.7 s


In [ ]:
%%time
nt=len(time)
nwaves, ni, nj = analysis.shape
for k in np.arange(len(time)):
    ttide = wt.tide_from_mapping(
    time[k].astype('datetime64[s]').astype('float64'),
    analysis.reshape(nwaves, ni*nj)).reshape(ni, nj)
    ttide_da=xr.DataArray(ttide,dims={'y','x'},name='tide')
    ttide_da.to_netcdf(path='/scratch/cnt0024/hmg2840/albert7a/eNATL60/eNATL60-BLBT02-S/1h/tide/pytide/eNATL60tiersX1-BLBT02_tide_y2009m07-m09_k'+str(k)+'.nc',engine='scipy')

In [ ]:
cluster.close()